# Organize and preprocessing data from matlab files

In [1]:
import numpy as np
import scipy.io

In [ ]:
data_structure_file = 'D:\data_structure_ANM210862.tar\data_structure_ANM210862\data_structure_ANM210862_20130628.mat'
label_to_save = 'ANM210862_20130628_labels.npy'
trace_dir = 'D:\\voltage_traces_ANM210862_20130628\\voltage_traces_ANM210862_20130628\\raw_trace_958_trial_'
sparse_epoch_to_save = 'sparse_ANM210862_20130628.npy'

In [ ]:
mat_dict = scipy.io.loadmat(data_structure_file)


data = mat_dict['obj']
data = data['trialTypeMat'][0,0]

valid_trials = []
labels = []
for i in range(len(data[0,:])):
    if data[1,i] == 1:
        labels.append(1)
        valid_trials.append(i+1)
    elif data[0,i] == 1:
        labels.append(0)
        valid_trials.append(i+1)
        
print(valid_trials)
print(labels)
# 1 means right, 0 menas left


file = open(label_to_save, 'wb')
np.save(file, labels)
file.close()


In [ ]:
# raw_trace_127_trial_5.mat

epochs = []
epochs = np.asarray(epochs)
min_timepoint = 1000000

for t in valid_trials:
    print(t)
    trial = scipy.io.loadmat(trace_dir + str(t) + '.mat')['ch_MUA']
    timepoint, chan = trial.shape
    if min_timepoint > timepoint:
        min_timepoint = timepoint
    
    trial = trial[:min_timepoint, :]
    trial = trial.reshape((1, min_timepoint, chan))
    print(trial.shape)
    if epochs.size == 0:
        epochs = trial
    else:
        epochs = epochs[:, :min_timepoint, :]
        epochs = np.concatenate((epochs, trial), axis = 0)
    print(epochs.shape)




In [ ]:
trial, timepoint, channel = epochs.shape
sparse_ind = [i*100 for i in range(timepoint//100) ]
print(len(sparse_ind))
sparse_epochs = epochs[:, sparse_ind, :]
sparse_epochs = np.swapaxes(sparse_epochs,1,2)


file = open(sparse_epoch_to_save, 'wb')
np.save(file, sparse_epochs)
file.close()

# Loading Data

In [2]:
sparse_epochs = np.load('sparse_ANM210861_20130702.npy')
labels = np.load('ANM210861_20130702_labels.npy')
print(sparse_epochs.shape)

# ind = np.linspace(0,3443-1, num = 300)
# ind = [int(np.floor(i)) for i in ind]
# sparse_epochs = sparse_epochs[:,:, ind]
# print(sparse_epochs.shape)



#get rid of flat channel

trial, channel,timepoint = sparse_epochs.shape
flat_chan = [i for i in range(channel) if sparse_epochs[0,i,0] == 0]
good_chan = [i for i in range(0,32) if i not in flat_chan]
print(flat_chan)
sparse_epochs = sparse_epochs[:,good_chan,:]
print(sparse_epochs.shape)


#standardize signal based on each channel

from sklearn.preprocessing import StandardScaler
import copy

epochs_std = copy.copy(sparse_epochs)
sample_num, chan_num, timepoint = sparse_epochs.shape
for c in range(chan_num):
    original_timepoints = sparse_epochs[:,c,:]
    scaler = StandardScaler()
    scaler.fit(original_timepoints)
    chan_std = scaler.transform(original_timepoints)
    epochs_std[:,c,:] = chan_std

(317, 32, 1033)
[0, 2, 3, 7, 10, 14, 15, 20, 22, 23, 25, 27, 30, 31]
(317, 18, 1033)


# Prediction on single session data

In [8]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = epochs_std

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]


    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    print(cov_X_train.shape)
    cov_X_test = cov.transform(X_test)
    
#     TSclassifier = pyriemann.classification.TSclassifier(metric='riemann', clf=sklearn.discriminant_analysis.LinearDiscriminantAnalysis())
    TSclassifier = pyriemann.classification.TSclassifier(metric='riemann')

    TSclassifier.fit(cov_X_train, y_train)
    
    
    y_predict = TSclassifier.predict(cov_X_test)
    print(sklearn.metrics.accuracy_score(TSclassifier.predict(cov_X_train), y_train))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

(253, 18, 18)
0.924901185770751
(253, 18, 18)
0.9209486166007905
(253, 18, 18)
0.9288537549407114
(253, 18, 18)
0.9288537549407114
(253, 18, 18)
0.9169960474308301
(253, 18, 18)
0.9367588932806324
(253, 18, 18)
0.932806324110672
(253, 18, 18)
0.9367588932806324
(253, 18, 18)
0.9367588932806324
(253, 18, 18)
0.924901185770751
Classification accuracy: 0.901563 / Chance level: 0.457413


#  PCA across trials

In [ ]:
import sklearn
from sklearn.decomposition import PCA
from sklearn import linear_model, decomposition, datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit  
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

for n_component in range(30,31):
    sparse_epochs_rs = sparse_epochs.reshape((trial, -1))
    pca = PCA(n_components=n_component)
    pca.fit(sparse_epochs_rs)
    total_variance = np.asarray(pca.explained_variance_ratio_)
    print(sum(total_variance))
    new_X = pca.transform(sparse_epochs_rs)

    samples = new_X

    scores = []
    cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
    for train_idx, test_idx in cv.split(samples):
        y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
        X_train = samples[train_idx]
        X_test = samples[test_idx]



#         logistic = linear_model.LogisticRegression(C = 1e-5)
#         logistic.fit(X_train, y_train)
        lda = LinearDiscriminantAnalysis()
        lda.fit(X_train, y_train)



#         y_predict = logistic.predict(X_test)
        y_predict = lda.predict(X_test)
        scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


    class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                              class_balance))

#  PCA across channels

In [ ]:
import sklearn
from sklearn.decomposition import PCA
from sklearn import linear_model, decomposition, datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit  

for n_components in range(3,12):
    samples = []
    for s in range(sparse_epochs.shape[0]):
        X = sparse_epochs[s,:,:]
        pca = PCA(n_components=n_components)
        pca.fit(X)
        new_X = pca.transform(X)
        samples.append(new_X)
    samples = np.asarray(samples)
    print(samples.shape)
    samples = samples.reshape((sparse_epochs.shape[0], -1))
    print(samples.shape)


    scores = []
    cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
    for train_idx, test_idx in cv.split(samples):
        y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
        X_train = samples[train_idx]
        X_test = samples[test_idx]



    #         logistic = linear_model.LogisticRegression(C = 1e-5)
    #         logistic.fit(X_train, y_train)
        lda = LinearDiscriminantAnalysis()
        lda.fit(X_train, y_train)



    #         y_predict = logistic.predict(X_test)
        y_predict = lda.predict(X_test)
        scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


    class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                              class_balance))

# CSP + LDA

In [69]:
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, find_events
from mne.channels import read_layout
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

#you need to make sure there is no flat channel! otherwise covariance matrix is not positive definite

scores = []
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
for train_idx, test_idx in cv.split(epochs_std):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_std[train_idx]
    X_test = epochs_std[test_idx]
    print(X_train.shape)
    
    csp = CSP(n_components=4, reg=None, log=True, norm_trace=False, cov_est = 'epoch', cov_method_params= 'shrinkage')
    new_epochs = csp.fit_transform(X_train,  y_train)
    print(new_epochs.shape)
    lda = LinearDiscriminantAnalysis()
    lda.fit(new_epochs, y_train)
    
#     logistic = linear_model.LogisticRegression(C = 1e-5)
#     logistic.fit(new_epochs, y_train)
    

    y_pred_csp = csp.transform(X_test)
    y_pred = lda.predict(y_pred_csp)
#     y_predict = logistic.predict(y_pred_csp)
    
    print(y_pred)
    print(y_test)
    print(sklearn.metrics.accuracy_score(y_pred, y_test))
    scores.append(sklearn.metrics.accuracy_score(y_pred, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

(262, 19, 300)
(262, 4)
[1 0 1 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 1 1 1 1 0 0 1 0 0 0 0 0 0 1 1 0 1 1 1
 0 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 1 0 1 0 0 0 1 1 1]
[1 0 1 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 1 0 0 1 1 1
 0 0 0 1 0 1 0 1 1 0 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 1 1 1]
0.803030303030303
(262, 19, 300)
(262, 4)
[1 1 0 1 1 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 1 1 1 1 0 0 0
 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 1 0 0 1 1 0 1 1 0 1 0 0]
[1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 1 1 1 0 0 0
 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 1 0 0 1 1 1 0 1 0 0 1 0]
0.8787878787878788
(262, 19, 300)
(262, 4)
[1 1 1 1 1 1 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 1 1 1 1 1 0 1
 0 1 1 1 1 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 0 1 0 1 1 1 0 0]
[1 1 0 1 1 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 1 0 1
 0 0 1 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0]
0.8636363636363636
(262, 19, 300)
(262, 4)
[0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 1 0 1 1

#  Aggregate data across session

In [2]:
from sklearn.preprocessing import StandardScaler
import copy

# sparse_files = ['sparse_ANM210861_20130701.npy', 'sparse_ANM210861_20130702.npy', 
#                'sparse_ANM210861_20130703.npy']

# label_files = ['ANM210861_20130701_labels.npy', 'ANM210861_20130702_labels.npy',
#               'ANM210861_20130703_labels.npy']

sparse_files = ['sparse_ANM210862_20130626.npy', 'sparse_ANM210862_20130627.npy', 
               'sparse_ANM210862_20130628.npy']

label_files = ['ANM210862_20130626_labels.npy', 'ANM210862_20130627_labels.npy',
              'ANM210862_20130628_labels.npy']


agg_spares_epochs = np.load(sparse_files[0])
agg_labels = np.load(label_files[0])
# ind = np.linspace(0,3443-1, num = 1033)
# ind = [int(np.floor(i)) for i in ind]
# agg_spares_epochs = agg_spares_epochs[:,:, ind]


for i in range(1,3):
    new_epochs =  np.load(sparse_files[i])
    agg_spares_epochs = np.concatenate((agg_spares_epochs, new_epochs), axis = 0)
    new_label = np.load(label_files[i])
    agg_labels = np.concatenate((agg_labels, new_label), axis = 0)

print(agg_spares_epochs.shape)
print(agg_labels.shape)



agg_epochs_std = copy.copy(agg_spares_epochs)
sample_num, chan_num, timepoint = agg_spares_epochs.shape
for c in range(chan_num):
    original_timepoints = agg_spares_epochs[:,c,:]
    scaler = StandardScaler()
    scaler.fit(original_timepoints)
    chan_std = scaler.transform(original_timepoints)
    agg_epochs_std[:,c,:] = chan_std



(642, 32, 1033)
(642,)


#  Prediction using Riemannian based classifier on mix dataset across session from same subject

In [29]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  
from scipy.fftpack import fft, ifft

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = agg_epochs_std
labels = agg_labels

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
#     TSclassifier = pyriemann.classification.TSclassifier(metric='riemann', clf=sklearn.discriminant_analysis.LinearDiscriminantAnalysis())
    TSclassifier = pyriemann.classification.TSclassifier(metric='riemann', clf=SVC(kernel='rbf', random_state=0, gamma= 0.03, C=100))
     
#     TSclassifier = pyriemann.classification.TSclassifier(metric='riemann', clf=LogisticRegression())

    TSclassifier.fit(cov_X_train, y_train)
    
    
    y_predict = TSclassifier.predict(cov_X_test)
    print(sklearn.metrics.accuracy_score(TSclassifier.predict(cov_X_train), y_train))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

(513, 32, 1033)
0.9961013645224172
(513, 32, 1033)
0.9980506822612085
(513, 32, 1033)
0.9980506822612085
(513, 32, 1033)
0.9941520467836257
(513, 32, 1033)
1.0
(513, 32, 1033)
1.0
(513, 32, 1033)
0.9980506822612085
(513, 32, 1033)
0.9961013645224172
(513, 32, 1033)
0.9980506822612085
(513, 32, 1033)
0.9980506822612085
Classification accuracy: 0.748837 / Chance level: 0.526480


#  Prediction using PCA + Logistic regression

In [ ]:
import sklearn
from sklearn.decomposition import PCA
from sklearn import linear_model, decomposition, datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit  

trial,_,_ = agg_spares_epochs.shape
for n_component in range(10,30):
    epochs_reshaped = agg_spares_epochs.reshape((trial, -1))
    pca = PCA(n_components=n_component)
    samples = pca.fit_transform(epochs_reshaped)
    total_variance = np.asarray(pca.explained_variance_ratio_)
    print(sum(total_variance))
    

    scores = []
    cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
    for train_idx, test_idx in cv.split(samples):
        y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
        X_train = samples[train_idx]
        X_test = samples[test_idx]



#         logistic = linear_model.LogisticRegression(C = 1e-5)
#         logistic.fit(X_train, y_train)
        lda = LinearDiscriminantAnalysis()
        lda.fit(X_train, y_train)



#         y_predict = logistic.predict(X_test)
        y_predict = lda.predict(X_test)
        scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


    class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                              class_balance))

#  Prediction Using PCA + logistic across channels on aggregated dataset

In [ ]:
for n_components in range(3,30):
    samples = []
    for s in range(agg_spares_epochs.shape[0]):
        X = agg_spares_epochs[s,:,:]
        pca = PCA(n_components=n_components)
        pca.fit(X)
        new_X = pca.transform(X)
        samples.append(new_X)
    samples = np.asarray(samples)
    print(samples.shape)
    samples = samples.reshape((agg_spares_epochs.shape[0], -1))

    print(samples.shape)


    labels = agg_labels
    scores = []
    cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
    for train_idx, test_idx in cv.split(samples):
        y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
        X_train = samples[train_idx]
        X_test = samples[test_idx]



    #         logistic = linear_model.LogisticRegression(C = 1e-5)
    #         logistic.fit(X_train, y_train)
        lda = LinearDiscriminantAnalysis()
        lda.fit(X_train, y_train)



    #         y_predict = logistic.predict(X_test)
        y_predict = lda.predict(X_test)
        scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


    class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
    print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                              class_balance))

# Boosting With MDM classifier 

In [17]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  

from sklearn.ensemble import AdaBoostClassifier #For Classification



cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = sparse_epochs

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    
    MDMclassifier = pyriemann.classification.MDM(metric='riemann')
    clf = AdaBoostClassifier(n_estimators=4, base_estimator=MDMclassifier,learning_rate=0.7)
    
    
    clf.fit(cov_X_train, y_train)
    
    
    y_predict = clf.predict(cov_X_test)
    print(sklearn.metrics.accuracy_score(clf.predict(cov_X_train), y_train))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))


#MDM with no boosting: 77%

(262, 19, 300)
0.7824427480916031
(262, 19, 300)
0.7977099236641222
(262, 19, 300)
0.7862595419847328
(262, 19, 300)
0.7938931297709924
(262, 19, 300)
0.8053435114503816
(262, 19, 300)
0.7786259541984732
(262, 19, 300)
0.7900763358778626
(262, 19, 300)
0.7633587786259542
(262, 19, 300)
0.7938931297709924
(262, 19, 300)
0.8091603053435115
Classification accuracy: 0.751515 / Chance level: 0.509146


In [23]:
import sklearn 
import pyriemann.estimation
import pyriemann.classification
import sklearn.linear_model
from sklearn.model_selection import ShuffleSplit  

from sklearn.ensemble import AdaBoostClassifier #For Classification
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC



cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []

epochs_data = agg_epochs_std
labels = agg_labels


for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    
#     TSclassifier = pyriemann.classification.TSclassifier(metric='riemann')
    TSclassifier = pyriemann.classification.TSclassifier(metric='riemann', clf=SVC(kernel='rbf', random_state=0, gamma= 0.03, C=100, probability=True))

    clf = AdaBoostClassifier(n_estimators=5, base_estimator=TSclassifier,learning_rate=1)
#     
    clf.fit(cov_X_train, y_train)
    
    
    y_predict = clf.predict(cov_X_test)
    print(sklearn.metrics.accuracy_score(clf.predict(cov_X_train), y_train))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))


class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))


#MDM with no boosting: 77%

(513, 32, 1033)
0.9961013645224172
(513, 32, 1033)
0.9980506822612085
(513, 32, 1033)
0.9961013645224172
(513, 32, 1033)
0.9941520467836257
(513, 32, 1033)
0.9980506822612085
(513, 32, 1033)
1.0
(513, 32, 1033)


KeyboardInterrupt: 

#  Bagging

In [22]:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = agg_epochs_std
labels = agg_labels
for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    n_samples, _,_ = cov_X_train.shape
    ind_set_1 = np.random.randint(0, n_samples, size = n_samples//3 * 2)
    ind_set_2 = np.random.randint(0, n_samples, size = n_samples//3 * 2)
    ind_set_3 = np.random.randint(0, n_samples, size = n_samples//3*2)

    clf_1 = pyriemann.classification.TSclassifier(metric='riemann', clf=SVC(kernel='rbf', random_state=0, gamma= 0.03, C=100, probability=True))
    clf_2 = pyriemann.classification.TSclassifier(metric='riemann', clf=SVC(kernel='rbf', random_state=0, gamma= 0.03, C=100, probability=True))
    clf_3 = pyriemann.classification.TSclassifier(metric='riemann', clf=SVC(kernel='rbf', random_state=0, gamma= 0.03, C=100, probability=True))

    clf_1.fit(cov_X_train[ind_set_1,:,:], y_train[ind_set_1])
    clf_2.fit(cov_X_train[ind_set_2,:,:], y_train[ind_set_2])
    clf_3.fit(cov_X_train[ind_set_3,:,:], y_train[ind_set_3])

    y_predict = scipy.stats.mode([clf_1.predict(cov_X_test), clf_2.predict(cov_X_test), clf_3.predict(cov_X_test)], axis=0).mode[0]
    print(sklearn.metrics.accuracy_score(scipy.stats.mode([clf_1.predict(cov_X_train), clf_2.predict(cov_X_train), clf_3.predict(cov_X_train)], axis=0).mode[0], y_train))
    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))

class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))



(513, 32, 1033)
0.8966861598440545
(513, 32, 1033)
0.9161793372319688
(513, 32, 1033)
0.8927875243664717
(513, 32, 1033)
0.8947368421052632
(513, 32, 1033)
0.8947368421052632
(513, 32, 1033)
0.8947368421052632
(513, 32, 1033)
0.8732943469785575
(513, 32, 1033)
0.9181286549707602
(513, 32, 1033)
0.8947368421052632
(513, 32, 1033)
0.9122807017543859
Classification accuracy: 0.730233 / Chance level: 0.526480


In [ ]:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
scores = []
epochs_data = agg_epochs_std
labels = agg_labels

n_estimator = 51
subset_size = n_samples//2

for train_idx, test_idx in cv.split(epochs_data):
    y_train, y_test = np.asarray(labels)[train_idx], np.asarray(labels)[test_idx]
    X_train = epochs_data[train_idx]
    X_test = epochs_data[test_idx]
    print(X_train.shape)

    
    cov =  pyriemann.estimation.Covariances('lwf')
    cov_X_train = cov.transform(X_train)
    cov_X_test = cov.transform(X_test)
    
    n_samples, _,_ = cov_X_train.shape
    
    ind_sets = [np.random.randint(0, n_samples, size = subset_size) for i in range(n_estimator)]
    
    clfs = [pyriemann.classification.TSclassifier(metric='riemann', clf=SVC(kernel='rbf', random_state=0, gamma= 0.03, C=100, probability=True)) for i in range(n_estimator)]

    for i in range(n_estimator):
        clfs[i].fit(cov_X_train[ind_sets[i],:,:], y_train[ind_sets[i]])
    
    y_predict = [clfs[i].predict(cov_X_test) for i in range(n_estimator)]
    y_predict = scipy.stats.mode(y_predict, axis=0).mode[0]
    

    scores.append(sklearn.metrics.accuracy_score(y_predict, y_test))
    print(scores)

class_balance = np.mean(np.asarray(labels) == np.asarray(labels)[0])
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))



(513, 32, 1033)
[0.7131782945736435]
(513, 32, 1033)
[0.7131782945736435, 0.7751937984496124]
(513, 32, 1033)
[0.7131782945736435, 0.7751937984496124, 0.7286821705426356]
(513, 32, 1033)
[0.7131782945736435, 0.7751937984496124, 0.7286821705426356, 0.7131782945736435]
(513, 32, 1033)
[0.7131782945736435, 0.7751937984496124, 0.7286821705426356, 0.7131782945736435, 0.7441860465116279]
(513, 32, 1033)
[0.7131782945736435, 0.7751937984496124, 0.7286821705426356, 0.7131782945736435, 0.7441860465116279, 0.7674418604651163]
(513, 32, 1033)
[0.7131782945736435, 0.7751937984496124, 0.7286821705426356, 0.7131782945736435, 0.7441860465116279, 0.7674418604651163, 0.7751937984496124]
(513, 32, 1033)
[0.7131782945736435, 0.7751937984496124, 0.7286821705426356, 0.7131782945736435, 0.7441860465116279, 0.7674418604651163, 0.7751937984496124, 0.7596899224806202]
(513, 32, 1033)


In [27]:
ind_sets = [np.random.randint(0, n_samples, size = n_samples//3 * 2) for i in range(5)]



342
